In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [104]:
import pandas as pd
import numpy as _np

In [113]:
import warnings

In [114]:
warnings.filterwarnings('error')

In [93]:
m = 4

x = concatenate((poisson(10, 100), poisson(20, 100), poisson(30, 100), poisson(40, 100)))

_lambda = linspace(x.min(), x.max(), 4, endpoint=True)
_gamma = array([[.6, .2, .15, .05],
                [.05, .6, .2, .15],
                [.15, .05, .6, .2],
                [.2, .15, .05, .6]])
_delta = repeat(1/m, m)

In [101]:
from fwbw import forward_backward1
from fwbw import forward_backward2


In [108]:
from scipy.special import logsumexp as _logsumexp


def EM(x, m, theta, meth, maxiter=1000, tol=1e-6):
    """Estimate the parameters of an m-state PoissonHMM.

    Params:
        x           (np.ndarray)    One-dimensional array of integer values.
        theta       (tuple)         Initial guesses (lambda, gamma, delta).
        maxiter     (int)           Mmaximum number of EM iterations.
        tol         (float)         Convergence criterion.
    """
    n = len(x)

    this_lambda = theta[0].copy()
    this_gamma = theta[1].copy()
    this_delta = theta[2].copy()

    next_lambda = theta[0].copy()
    next_gamma = theta[1].copy()
    next_delta = theta[2].copy()

    for i in range(maxiter):
        alpha, beta, prob = meth(x, m, this_lambda, this_gamma, this_delta)

        c = max(alpha[-1])
        log_likelihood = c + _logsumexp(alpha[-1] - c)

        for j in range(m):
            for k in range(m):
                next_gamma[j, k] *= _np.sum(_np.exp(alpha[:n-1, j] +
                                            beta[1:n, k] +
                                            prob[1:n, k] -
                                            log_likelihood))
        next_gamma /= _np.sum(next_gamma, axis=1)

        rab = _np.exp(alpha + beta - log_likelihood)
        next_lambda = (rab * x[:, None]).sum(axis=0) / rab.sum(axis=0)

        next_delta = rab[0] / rab[0].sum()

        crit = (_np.abs(this_lambda - next_lambda).sum() +
                _np.abs(this_gamma - next_gamma).sum()  +
                _np.abs(this_delta - next_delta).sum())

        if crit < tol:
            return this_lambda, this_gamma, this_delta, i, log_likelihood

        this_lambda = next_lambda.copy()
        this_gamma = next_gamma.copy()
        this_delta = next_delta.copy()

    return False



In [116]:
out = EM(x, m, (_lambda, _gamma, _delta), forward_backward1)

[4.69823803e-04 9.99407157e-01 1.23018020e-04 6.94159073e-10]
[2.82557866e-05 9.99971743e-01 1.01087810e-09 8.65433751e-18]
[2.02266737e-06 9.99997977e-01 8.11323256e-15 1.18436236e-26]
[1.99746148e-06 9.99998003e-01 5.52751427e-20 6.43414717e-36]
[2.12895256e-05 9.99978710e-01 2.49772607e-25 2.11337553e-45]
[1.38069451e-03 9.98619305e-01 1.07847381e-30 6.02474165e-55]
[1.71342419e-01 8.28657581e-01 5.99812401e-36 1.50157070e-64]
[9.73947735e-01 2.60522647e-02 2.10830913e-42 1.05183353e-75]
[9.99987169e-01 1.28307138e-05 1.08704099e-50 3.74250609e-89]
[1.00000000e+000 1.30612649e-010 6.62880952e-061 4.19452857e-105]
[1.00000000e+000 2.74489739e-017 4.36897270e-073 6.47451439e-124]
[1.00000000e+000 1.20163175e-025 3.06714357e-087 5.62550389e-146]
[1.00000000e+000 1.10392399e-035 2.20686680e-103 1.78053083e-171]
[1.00000000e+000 2.13725545e-047 1.57795267e-121 2.12422476e-200]
[1.00000000e+000 8.73895146e-061 1.10185549e-141 1.08426502e-232]
[1.00000000e+000 7.55438385e-076 7.44875390e-1

RuntimeWarning: divide by zero encountered in log

In [112]:
out = EM(x, m, (_lambda, _gamma, _delta), forward_backward2)


/Users/michael/devel/repos/apollon/apollon/hmm/fwbw.py:75: RuntimeWarning: divide by zero encountered in log
  alpha[0] = _np.log(a_t) + lscale
/Users/michael/devel/repos/apollon/apollon/hmm/fwbw.py:83: RuntimeWarning: divide by zero encountered in log
  alpha[i] = _np.log(a_t) + lscale
